In [1]:
from dotenv import load_dotenv
import os
from hera.workflows import EmptyDirVolume, models, script, Artifact, Parameter, DAG, Steps, Step, NoneArchiveStrategy, Workflow
from hera.shared import global_config

load_dotenv("/home/otto/s1_zarr/.env")

True

In [2]:
global_config.host = "https://dev.services.eodc.eu/workflows/"
global_config.namespace = "inca"
global_config.token = os.getenv("argo_token")
global_config.image = "ghcr.io/oscipal/image_zarr:latest"

In [3]:
empty_dir = [models.Volume(name="emptydir", empty_dir=models.EmptyDirVolumeSource())]

In [ ]:
@script(volume_mounts=[models.VolumeMount(name="emptydir", mount_path="/mnt/vol/")])
def download_and_use():
    from urllib.request import urlretrieve
    import datetime
    import os
    import xarray as xr

    mount_path = "/mnt/vol/"

    url = f"https://public.hub.geosphere.at/datahub/resources/inca-v1-1h-1km/filelisting/RR/INCAL_HOURLY_RR_202506.nc"
    urlretrieve(url, os.path.join(mount_path, "INCA.nc"))

    ds = xr.open_dataset(os.path.join(mount_path, "INCA.nc")).isel(time=1, x=1, y=1)["RR"].load()
    print(ds.values)

In [15]:
with Workflow(
    generate_name="empty-dir-",
    volumes = empty_dir,
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        download_and_use()

In [16]:
w.create()

Workflow(api_version=None, kind=None, metadata=ObjectMeta(annotations=None, cluster_name=None, creation_timestamp=Time(__root__=datetime.datetime(2025, 7, 24, 7, 11, 16, tzinfo=datetime.timezone.utc)), deletion_grace_period_seconds=None, deletion_timestamp=None, finalizers=None, generate_name='empty-dir-', generation=1, labels={'workflows.argoproj.io/creator': 'system-serviceaccount-default-jenkins'}, managed_fields=[ManagedFieldsEntry(api_version='argoproj.io/v1alpha1', fields_type='FieldsV1', fields_v1=FieldsV1(), manager='argo', operation='Update', subresource=None, time=Time(__root__=datetime.datetime(2025, 7, 24, 7, 11, 16, tzinfo=datetime.timezone.utc)))], name='empty-dir-bq4d2', namespace='inca', owner_references=None, resource_version='315324681', self_link=None, uid='9c77034d-b230-4ea2-aa6f-843a878e2284'), spec=WorkflowSpec(active_deadline_seconds=None, affinity=None, archive_logs=None, arguments=Arguments(artifacts=None, parameters=None), artifact_gc=None, artifact_repository